In [36]:
import os
import pandas as pd
import pyarrow.parquet as pq
from IPython.display import display, HTML

In [37]:
dq_results = []
next_number = 1  # Initial value for the sequential number

In [38]:
# read parquet files from foldaer as dataframe
def read_parquet_from_folder(directory):
    files = [f for f in os.listdir(directory) if f.endswith('.parquet')]
    dfs = []
    for file in files:
        file_path = os.path.join(directory, file)
        table = pq.read_table(file_path)
        df = table.to_pandas()
        dfs.append(df)
    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

In [39]:
# add to result list result of check
def add_to_results(res, columns, table_name, check_type):
    global next_number 
    if res.empty:

        results = {
            "#": str(next_number),
            "Table": table_name,
            "DQ Check": check_type,
            "Columns": columns,
            "Status": "Passed",
            "Bad Data": ""
        }
    else:
        res = res.copy()
        for col in res.columns:
            if res[col].dtype != 'object':
                res.loc[:, col] = res.loc[:, col].astype(str)
        results =  {
            "#": str(next_number),
            "Table": table_name,
            "DQ Check": check_type,
            "Columns": columns,
            "Status": "Failed",
            "Bad Data":res.to_dict('records')
        }

    dq_results.append(results)
    next_number += 1  # Increment the sequential number for the next call

Before running an abligatory list of check I decided to implement some general checks

In [40]:
# Function to check if all columns are present and have required column names
def check_columns_present(parquet_path, columns, table_name):
    df = read_parquet_from_folder(parquet_path)
    missing_columns = set(columns) - set(df.columns)
    global next_number 
    if not missing_columns:
        results = {
            "#": str(next_number),
            "Table": table_name,
            "DQ Check": "Completeness",
            "Columns":  columns,
            "Status": "Passed",
            "Bad Data": ""
        }
    else:
        results =  {
            "#": str(next_number),
            "Table": table_name,
            "DQ Check": "Completeness",
            "Columns":  columns,
            "Status": "Failed",
            "Bad Data": "Missing columns: " + ", ".join(missing_columns)
        }
    dq_results.append(results)
    next_number += 1  # Increment the sequential number for the next call

In [41]:
check_columns_present('/home/datalab-user/raw/carriers/', ["code", "description"],"Carriers" )
check_columns_present('/home/datalab-user/raw/airports/', ["iata", "airport", "city", "state", "country", "lat", "long"], "Airports")
check_columns_present('/home/datalab-user/raw/flights/', ["Year","Month","DayofMonth","DayOfWeek","DepTime","CRSDepTime","ArrTime","CRSArrTime","UniqueCarrier","FlightNum","TailNum","ActualElapsedTime","CRSElapsedTime","AirTime","ArrDelay","DepDelay","Origin","Dest","Distance","TaxiIn","TaxiOut","Cancelled","CancellationCode","Diverted","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay"], "Flights")

Obligatory part:  
Carrier: completeness by not nullable fields  
Airports: Uniqueness by PK  
Flights: Consistency check for UniqueCarrier  
Flights: Consistency check for DepDelay  

In [42]:
# Carrier: completeness by not nullable fields
# Function to check completeness
def check_completeness(parquet_path, columns, table_name):
    df = read_parquet_from_folder(parquet_path)
    null_values = df[df[columns].isnull().any(axis=1)]
    add_to_results(null_values,  columns, table_name, "Completeness" )

In [43]:
check_completeness('/home/datalab-user/raw/carriers/', ["code", "description"], "Carriers")

In [44]:
#Airports: Uniqueness by PK
# Function to check uniqueness
def check_uniqueness(parquet_path, columns, table_name):
    df = read_parquet_from_folder(parquet_path)
    duplicates = df[df.duplicated(subset=columns, keep=False)]
    # Select only the columns of interest for better readability in our report
    duplicates_filtered = duplicates[columns]
    add_to_results(duplicates_filtered, columns, table_name, "Uniqueness")
    

In [45]:
check_uniqueness('/home/datalab-user/raw/airports/', ["iata"], "Airports")

In [46]:
#Flights: Consistency check for UniqueCarrier
# Function to check consistency refferences
def check_consistency_ref(link_path, refs_path, link_column, ref_column, table_name):
    link_df = read_parquet_from_folder(link_path)
    ref_df = read_parquet_from_folder(refs_path)
    
    invalid_values = ref_df[~ref_df[ref_column].isin(link_df[link_column])]
    invalid_values_filtered = pd.DataFrame(invalid_values[ref_column])  
    add_to_results(invalid_values_filtered, ref_column, table_name, "Consistency")

In [47]:
check_consistency_ref('/home/datalab-user/raw/carriers/', '/home/datalab-user/raw/flights/', "code", "UniqueCarrier", "Flights")

In [48]:
# Flights: Consistency check for DepDelay
# Function to check if values in specified columns match the given calculation
def check_time_values(parquet_path, column1, column2, column3, table_name):
    df = read_parquet_from_folder(parquet_path)
    df[column1] = pd.to_numeric(df[column1], errors='coerce')
    df[column2] = df[column2].astype(str).str.zfill(4)  # Pad the hour-minute values with leading zeros
    df[column2] = pd.to_numeric(df[column2], errors='coerce')
    df[column3] = df[column3].astype(str).str.zfill(4)  # Pad the hour-minute values with leading zeros
    df[column3] = pd.to_numeric(df[column3], errors='coerce')
    
    # Calculate the expected time in minutes
    expected_time = (df[column2] // 100) * 60 + (df[column2] % 100) - (df[column3] // 100) * 60 - (df[column3] % 100)
    
    # Filter invalid values based on the calculation
    invalid_values = df[~(df[column1] == expected_time)]
    invalid_values_filtered = invalid_values[[column1, column2, column3]]  
    add_to_results(invalid_values_filtered, column1, table_name, "Consistency")

In [49]:
check_time_values('/home/datalab-user/raw/flights/', "DepDelay", "DepTime", "CRSDepTime", "Flights")

Run check similar to obligatory checks

In [50]:
# check completeness in all other tables
check_completeness('/home/datalab-user/raw/airports/', ["iata", "airport", "city", "state", "country", "lat", "longt"], "Airports")
check_completeness('/home/datalab-user/raw/flights/', ["Year","Month","DayofMonth","DayOfWeek","DepTime","CRSDepTime","ArrTime","CRSArrTime","UniqueCarrier","FlightNum","TailNum","ActualElapsedTime","CRSElapsedTime","AirTime","ArrDelay","DepDelay","Origin","Dest","Distance","TaxiIn","TaxiOut","Cancelled","Diverted","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay"], "Flights")


In [51]:
# check uniqueness by PK for all other tables
check_uniqueness('/home/datalab-user/raw/carriers/', ["code", "description"], "Carriers")
check_uniqueness('/home/datalab-user/raw/flights/', ["Year", "Month", "DayofMonth", "DepTime", "FlightNum" ], "Flights")

In [52]:
# check consistensy for all another check of this type
check_consistency_ref('/home/datalab-user/raw/airports/', '/home/datalab-user/raw/flights/', "iata", "Origin", "Flights")
check_consistency_ref('/home/datalab-user/raw/airports/', '/home/datalab-user/raw/flights/', "iata", "Dest", "Flights")

In [53]:
# check consistansy if values in specified columns match the given calculation  for all similar checks
check_time_values('/home/datalab-user/raw/flights/', "ActualElapsedTime", "ArrTime", "DepTime", "Flights")
check_time_values('/home/datalab-user/raw/flights/', "CRSElapsedTime", "CRSArrTime", "CRSDepTime", "Flights")
check_time_values('/home/datalab-user/raw/flights/', "ArrDelay", "ArrTime", "CRSArrTime", "Flights")

Implement a little bit more checks, Like non-obligatory part

In [54]:
# Function to check validity by length
def check_validity(parquet_path, column, max_length, table_name):
    df = read_parquet_from_folder(parquet_path)
    df[column] = df[column].astype(str)  # Convert the column values to string type
    invalid_values = df[df[column].str.len() > max_length]
    invalid_values_filtered = invalid_values[column]
    add_to_results(invalid_values_filtered, column, table_name, "Validity")


In [55]:
check_validity('/home/datalab-user/raw/carriers/', "code", 7,"Carriers")
check_validity('/home/datalab-user/raw/carriers/', "description", 100,"Carriers")
check_validity('/home/datalab-user/raw/airports/', "iata", 4, "Airports")
check_validity('/home/datalab-user/raw/airports/', "state", 2, "Airports")
check_validity('/home/datalab-user/raw/flights/', "Year", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "Month", 2, "Flights")
check_validity('/home/datalab-user/raw/flights/', "DayofMonth", 2, "Flights")
check_validity('/home/datalab-user/raw/flights/', "DayOfWeek", 1, "Flights")
check_validity('/home/datalab-user/raw/flights/', "DepTime", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "CRSDepTime", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "ArrTime", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "CRSArrTime", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "UniqueCarrier", 7, "Flights")
check_validity('/home/datalab-user/raw/flights/', "FlightNum", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "TailNum", 6, "Flights")
check_validity('/home/datalab-user/raw/flights/', "ActualElapsedTime", 3, "Flights")
check_validity('/home/datalab-user/raw/flights/', "CRSElapsedTime", 3, "Flights")
check_validity('/home/datalab-user/raw/flights/', "AirTime", 3, "Flights")
check_validity('/home/datalab-user/raw/flights/', "ArrDelay", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "DepDelay", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "Origin", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "Dest", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "Distance", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "TaxiIn", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "TaxiOut", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "Cancelled", 1, "Flights")
check_validity('/home/datalab-user/raw/flights/', "CancellationCode", 1, "Flights")
check_validity('/home/datalab-user/raw/flights/', "Diverted", 1, "Flights")
check_validity('/home/datalab-user/raw/flights/', "CarrierDelay", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "WeatherDelay", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "NASDelay", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "SecurityDelay", 4, "Flights")
check_validity('/home/datalab-user/raw/flights/', "LateAircraftDelay", 4, "Flights")

In [56]:
# Function to check if values are 'NA' or valid state code
def check_accuracy(parquet_path, column, table_name):
    df = read_parquet_from_folder(parquet_path)
    invalid_values = df[~df[column].isin(['NA', 'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT','NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'VI', 'WA', 'WV', 'WI', 'WY'])]
    invalid_values_filtered =  pd.DataFrame(invalid_values[column])
    add_to_results(invalid_values_filtered, column, table_name, "Accuracy")

In [57]:
check_accuracy('/home/datalab-user/raw/airports/', 'state', "Airports")

In [58]:
# Function to check consistency for state column
def check_consistency_state(parquet_path, country_column, state_column, table_name):
    df = read_parquet_from_folder(parquet_path)
    invalid_values = df[
        ((df[country_column] != 'USA') & (df[state_column] != 'NA')) |
        ((df[country_column] == 'USA') & (~df[state_column].isin(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT','NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'VI', 'WA', 'WV', 'WI', 'WY'])))
    ]
    invalid_values_filtered = pd.DataFrame(invalid_values[state_column])    
    add_to_results(invalid_values_filtered, state_column, table_name, "Consistency")

In [59]:
check_consistency_state('/home/datalab-user/raw/airports/', 'country', 'state', "Airports")

In [60]:
# Function to check validity for values in range
def check_valid_range(parquet_path, column, min_value, max_value, table_name):
    df = read_parquet_from_folder(parquet_path)

    # Convert column values to numeric type if possible
    df[column] = pd.to_numeric(df[column], errors='coerce')

    # Filter invalid values based on numeric comparison
    invalid_values = df[~df[column].between(min_value, max_value)]
    invalid_values_filtered = pd.DataFrame(invalid_values[column])   
    add_to_results(invalid_values_filtered, column, table_name, "Validity")


In [61]:
check_valid_range('/home/datalab-user/raw/airports/', 'lat', -90, 90, "Airports")
check_valid_range('/home/datalab-user/raw/airports/', 'longt', -180, 180, "Airports")

In [62]:
check_valid_range('/home/datalab-user/raw/flights/', 'DepTime', 0, 2359, "Flights")
check_valid_range('/home/datalab-user/raw/flights/', 'CRSDepTime', 0, 2359, "Flights")
check_valid_range('/home/datalab-user/raw/flights/', 'ArrTime',  0, 2359,"Flights")
check_valid_range('/home/datalab-user/raw/flights/', 'CRSArrTime', 0, 2359, "Flights")

In [63]:
def check_validity_binary(parquet_path, column, table_name):
    df = read_parquet_from_folder(parquet_path)
    numeric_values = pd.to_numeric(df[column], errors='coerce')
    invalid_values = df[~((numeric_values >= 0) & (numeric_values <= 1))]
    invalid_values_filtered = pd.DataFrame(invalid_values[column])  
    add_to_results(invalid_values_filtered, column, table_name, "Validity")

In [64]:
check_validity_binary('/home/datalab-user/raw/flights/', "Cancelled", "Flights")
check_validity_binary('/home/datalab-user/raw/flights/', "Diverted", "Flights")

In [65]:
def check_validity_ABC(parquet_path, column, table_name):
    df = read_parquet_from_folder(parquet_path)

    mask = (df['CancellationCode'].isin(['A', 'B', 'C'])) | (df['CancellationCode'].isnull()) | (df['CancellationCode'].isnull()) | (df['CancellationCode'] == '') 
    invalid_values = df[~mask]
    invalid_values_filtered = pd.DataFrame(invalid_values[column])     
    add_to_results(invalid_values_filtered, column, table_name, "Validity")

In [66]:
check_validity_ABC('/home/datalab-user/raw/flights/', "CancellationCode", "Flights")

In [67]:
def check_consistency_cancelled(parquet_path, cancelled_column, check_column, table_name):
    df = read_parquet_from_folder(parquet_path)
    mask = ((df[check_column].isin(['A', 'B', 'C'])) & (df[cancelled_column] == '1'))|((df[check_column].isnull()) | (df[check_column]=='') &(df[cancelled_column] == '0'))
    invalid_values = df[~mask]
    invalid_values_filtered = invalid_values[[cancelled_column, check_column]]   
    add_to_results(invalid_values_filtered, check_column, table_name, "Consistency")

In [68]:
check_consistency_cancelled('/home/datalab-user/raw/flights/', "Cancelled", "CancellationCode", "Flights")

Print all results, for better readability I also create HTML report  
Results of obligatory part rows 4-7

In [69]:
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(dq_results)
html_string = '''
<style>
table {
  text-align: left;
}
th, td {
  text-align: left;
}
</style>
'''

html_string += df.to_html(index=False)

# Save the HTML string to a file
with open('results_table.html', 'w') as file:
    file.write(html_string)

# Display the HTML table in the notebook
display(HTML(html_string))

#,Table,DQ Check,Columns,Status,Bad Data
1,Carriers,Completeness,"[code, description]",Passed,
2,Airports,Completeness,"[iata, airport, city, state, country, lat, long]",Failed,Missing columns: long
3,Flights,Completeness,"[Year, Month, DayofMonth, DayOfWeek, DepTime, CRSDepTime, ArrTime, CRSArrTime, UniqueCarrier, FlightNum, TailNum, ActualElapsedTime, CRSElapsedTime, AirTime, ArrDelay, DepDelay, Origin, Dest, Distance, TaxiIn, TaxiOut, Cancelled, CancellationCode, Diverted, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay]",Passed,
4,Carriers,Completeness,"[code, description]",Passed,
5,Airports,Uniqueness,[iata],Failed,"[{'iata': '00M'}, {'iata': '00R'}, {'iata': '00V'}, {'iata': ''}, {'iata': ''}, {'iata': 'Z08'}, {'iata': 'Z09'}, {'iata': 'Z13'}, {'iata': '00M'}, {'iata': '00R'}, {'iata': '00V'}, {'iata': 'Z08'}, {'iata': 'Z09'}, {'iata': 'Z13'}]"
6,Flights,Consistency,UniqueCarrier,Failed,"[{'UniqueCarrier': 'AXX'}, {'UniqueCarrier': 'AXX'}, {'UniqueCarrier': 'AXX'}, {'UniqueCarrier': '1B9'}, {'UniqueCarrier': '1B9'}, {'UniqueCarrier': '1B9'}, {'UniqueCarrier': '1B9'}, {'UniqueCarrier': '1B9'}, {'UniqueCarrier': '1B9'}]"
7,Flights,Consistency,DepDelay,Failed,"[{'DepDelay': '0.0', 'DepTime': 'nan', 'CRSDepTime': '1100'}, {'DepDelay': '0.0', 'DepTime': 'nan', 'CRSDepTime': '905'}, {'DepDelay': '0.0', 'DepTime': 'nan', 'CRSDepTime': '1620'}, {'DepDelay': '0.0', 'DepTime': 'nan', 'CRSDepTime': '1930'}, {'DepDelay': 'nan', 'DepTime': '0.0', 'CRSDepTime': '0'}, {'DepDelay': '80.0', 'DepTime': '0.0', 'CRSDepTime': '400'}, {'DepDelay': '80.0', 'DepTime': '520.0', 'CRSDepTime': '0'}, {'DepDelay': 'nan', 'DepTime': '1519.0', 'CRSDepTime': '1525'}, {'DepDelay': '6.0', 'DepTime': '859.0', 'CRSDepTime': '905'}, {'DepDelay': '-5.0', 'DepTime': '2500.0', 'CRSDepTime': '1505'}, {'DepDelay': '-6.0', 'DepTime': '1239.0', 'CRSDepTime': '1275'}, {'DepDelay': '0.0', 'DepTime': 'nan', 'CRSDepTime': '700'}]"
8,Airports,Completeness,"[iata, airport, city, state, country, lat, longt]",Passed,
9,Flights,Completeness,"[Year, Month, DayofMonth, DayOfWeek, DepTime, CRSDepTime, ArrTime, CRSArrTime, UniqueCarrier, FlightNum, TailNum, ActualElapsedTime, CRSElapsedTime, AirTime, ArrDelay, DepDelay, Origin, Dest, Distance, TaxiIn, TaxiOut, Cancelled, Diverted, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay]",Passed,
10,Carriers,Uniqueness,"[code, description]",Failed,"[{'code': '07Q', 'description': 'Flair Airlines Ltd.'}, {'code': '0BQ', 'description': 'DCA'}, {'code': 'ZUQ', 'description': 'Zuliana De Aviacion'}, {'code': '07Q', 'description': 'Flair Airlines Ltd.'}, {'code': '0BQ', 'description': 'DCA'}, {'code': '07Q', 'description': 'Flair Airlines Ltd.'}, {'code': 'ZUQ', 'description': 'Zuliana De Aviacion'}]"
